In [13]:
path_to_pt_file = '/Users/jongya/Desktop/Workspace/lab/20230210_sesac_final/yolov5/yolov5/best.pt'
path_to_weights_file = '/Users/jongya/Desktop/Workspace/lab/20230210_sesac_final/yolov5/yolov5/best.weights'

In [7]:
!git clone https://github.com/ultralytics/yolov5.git

fatal: 대상 경로가('yolov5') 이미 있고 빈 디렉터리가 아닙니다.


In [8]:
!cd ./yolov5

In [9]:
!pip install -r requirements.txt

  Using cached GitPython-3.1.31-py3-none-any.whl (184 kB)
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 11.5 MB/s eta 0:00:00
  Using cached idna-3.4-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [11]:
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 24.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.1
    Uninstalling protobuf-4.22.1:
      Successfully uninstalled protobuf-4.22.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 18.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 19.8 MB/s eta 0:00:00


In [14]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'custom', path=path_to_pt_file)

/Users/jongya/miniforge3/envs/sesac_final/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using cache found in /Users/jongya/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-3-3 Python-3.9.0 torch-1.13.1 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [31]:
import os
os.chdir('../')

In [32]:
import os
import shutil
import torch
from models.yolo import Model

os.chdir(os.getcwd() + '/yolov5')
shutil.copy('./models/yolov5s.yaml', './yolov5s.yaml')

model = Model()
model.load_state_dict(torch.load(path_to_pt_file, map_location=torch.device('cpu'))['model'].state_dict())
model.float()
model.eval()

x = torch.randn(1, 3, 416, 416)
torch.onnx.export(model, x, 'yolov5s.onnx', opset_version=12)

model.export = True
model(torch.zeros(1, 3, 416, 416))  # dry run
model.export = False

model.model[-1].export = True  # set Detect() layer export=True
model.model[-1].export_weights(path_to_weights_file)  # save as '.pt'


                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 2]              
  8                -1  1   1182720  

RuntimeError: Error(s) in loading state_dict for DetectionModel:
	size mismatch for model.24.m.0.weight: copying a param with shape torch.Size([24, 128, 1, 1]) from checkpoint, the shape in current model is torch.Size([255, 128, 1, 1]).
	size mismatch for model.24.m.0.bias: copying a param with shape torch.Size([24]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for model.24.m.1.weight: copying a param with shape torch.Size([24, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([255, 256, 1, 1]).
	size mismatch for model.24.m.1.bias: copying a param with shape torch.Size([24]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for model.24.m.2.weight: copying a param with shape torch.Size([24, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([255, 512, 1, 1]).
	size mismatch for model.24.m.2.bias: copying a param with shape torch.Size([24]) from checkpoint, the shape in current mod